## Core Tests

Test the functionality of the basic audio/MIDI handling features of the app.

- Loading audio/MIDI
- Playback audio/MIDI

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('..') # leave the notebooks folder into root as the main

from app.core.audio.AudioData import AudioData
from app.core.audio.AudioPlayer import AudioPlayer
from app.core.midi.MidiData import MidiData
from app.core.midi.MidiPlayer import MidiPlayer
from app.core.midi.MidiSynth import MidiSynth

## CORE: MidiData, MidiPlayer, MidiSynth
Load in a MIDI file and create a `MidiData` instance to turn it into a dataframe of messages to be parsed in other parts of the program.

You can play the MIDI to inspect that it sounds right by first loading a `MidiSynth` instance (+ providing a soundfont file), loading your MidiData into a `MidiPlayer` instance, and then running `play()`.

In [ ]:
# To inspect how MIDI file is currently parsed
MIDI_FILEPATH = '../data/fugue_midi.mid'
midi_data = MidiData(MIDI_FILEPATH)

# Initialize a MidiSynth + MidiPlayer with a given soundfont
SOUNDFONT = 'data/MuseScore_General.sf3'
midi_synth = MidiSynth(soundfont_path=SOUNDFONT)
midi_player = MidiPlayer(midi_synth)

# Play the midi file
midi_player.load_midi(midi_data)
midi_player.play(start_time=0)

## The MIDI data structures

We have the following data structures to store MIDI information
- message_dict
- program_dict
- pitch_df

### Details

`message_dict` is used directly in playing back the MIDI file with the MidiSynth. We interact with MIDI through the series of messages to play at given times. We have the following types of messages:
- program_change - changing 'instrument'
- control_change - changing volume(?)
- note_on - start playing a given note with some volume(velocity)
- note_off - stop playing the note with the given velocity

`program_dict` tells us at what point do we need to change 'programs' or 'instruments'.

`pitch_df` is like the interpretable dataframe of notes which make up the MIDI data. This is used as the input into creating the MIDI string for MIDI-audio alignment.

In [9]:
midi_data.message_dict

{0: [Message('control_change', channel=0, control=121, value=0, time=0),
  Message('program_change', channel=0, program=40, time=0),
  Message('control_change', channel=0, control=7, value=100, time=0),
  Message('control_change', channel=0, control=10, value=64, time=0),
  Message('control_change', channel=0, control=91, value=0, time=0),
  Message('control_change', channel=0, control=93, value=0, time=0),
  Message('note_on', channel=0, note=62, velocity=100, time=0),
  Message('control_change', channel=0, control=2, value=80, time=0),
  Message('control_change', channel=1, control=121, value=0, time=0),
  Message('program_change', channel=1, program=45, time=0),
  Message('control_change', channel=1, control=7, value=100, time=0),
  Message('control_change', channel=1, control=10, value=64, time=0),
  Message('control_change', channel=1, control=91, value=0, time=0),
  Message('control_change', channel=1, control=93, value=0, time=0),
  Message('control_change', channel=2, control=1

In [7]:
midi_data.program_dict

{0: Message('program_change', channel=0, program=40, time=0),
 1: Message('program_change', channel=1, program=45, time=0),
 2: Message('program_change', channel=2, program=44, time=0)}

In [8]:
midi_data.pitch_df

,note_idx,start,channel,pitch,velocity,duration,frequency
0,0,0.000000,0,62,100,0.185938,293.664768
1,1,0.187500,0,69,100,0.185938,440.000000
2,2,0.375001,0,73,100,0.185938,554.365262
3,3,0.562501,0,76,100,0.185938,659.255114
4,4,0.750002,0,77,100,0.185938,698.456463
...,...,...,...,...,...,...,...
60,60,11.250030,0,63,100,0.185938,311.126984
61,61,11.437530,0,74,100,0.185938,587.329536
62,62,11.625031,0,79,100,0.185938,783.990872
63,63,11.812532,0,72,100,0.185938,523.251131


## CORE: AudioData, AudioPlayer

Load in an audio file (.mp3) and create a `AudioData` instance to convert it into a numpy array of samples which can be played out using an `AudioPlayer` instance which uses the `sounddevice` package to play the audio in a separate thread.

Uses the sample rate configured in `app/config.py`, which is set to 44100 as default.

In [7]:
AUDIO_FILEPATH = 'data/audio_files/user_fugue2.mp3'
user_audio = AudioData(audio_filepath=AUDIO_FILEPATH, sample_rate=44100)

audio_player = AudioPlayer()
audio_player.load_audio_data(user_audio) # load the player
audio_player.play()